In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import graphviz

### Single Dataset

* load the dataset and remove the non-necessary columns

In [2]:
df = pd.read_csv("SingleDataset\data_for_Qair_pred.csv", delimiter=',')
df = df.drop(columns=['Unnamed: 0', 'lat', 'lon', 'geometry', 'index_right'])
df

,Fall_PSurf_mean,Spring_PSurf_mean,Summer_PSurf_mean,Winter_PSurf_mean,Fall_Qair_mean,Spring_Qair_mean,Summer_Qair_mean,Winter_Qair_mean,Fall_Rainf_mean,Spring_Rainf_mean,...,CEC topsoil,CEC subsoil,CEC clay topsoil,CEC Clay subsoil,CaCO3 % topsoil,CaCO3 % subsoil,BD topsoil,BD subsoil,C/N topsoil,C/N subsoil
0,0.715987,0.711952,0.711822,0.712600,0.167288,0.211318,0.453284,0.201368,0.000000,0.034467,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,0.740597,0.736342,0.736393,0.737106,0.153214,0.206133,0.436666,0.194863,0.000000,0.050400,...,0.220365,0.208388,0.876494,0.704225,0.390578,0.503876,0.815217,0.760870,0.384615,0.68750
2,0.734826,0.730549,0.730519,0.731318,0.154547,0.212596,0.423964,0.197594,0.000000,0.028436,...,0.220365,0.208388,0.876494,0.704225,0.390578,0.503876,0.815217,0.760870,0.384615,0.68750
3,0.675135,0.671223,0.670347,0.672390,0.128757,0.164859,0.386399,0.179622,0.000000,0.019903,...,0.131712,0.105513,0.365206,0.197183,0.001491,0.005700,0.326087,0.407609,0.264423,0.25000
4,0.703261,0.698991,0.698185,0.700375,0.122039,0.159077,0.360461,0.176640,0.000000,0.010883,...,0.131712,0.105513,0.365206,0.197183,0.001491,0.005700,0.326087,0.407609,0.264423,0.25000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,0.759708,0.768804,0.771940,0.765964,0.771392,0.794850,0.747227,0.793683,0.850751,0.757786,...,0.174519,0.327618,0.285525,0.504225,0.000894,0.002052,0.967391,0.967391,1.000000,0.75625
850,0.824687,0.834484,0.836913,0.830241,0.795485,0.797414,0.782831,0.822217,0.905015,0.761129,...,0.174519,0.327618,0.285525,0.504225,0.000894,0.002052,0.967391,0.967391,1.000000,0.75625
851,0.863529,0.874116,0.875757,0.868317,0.828288,0.793065,0.757406,0.868328,0.892714,0.818705,...,0.378419,0.374308,0.565737,0.595775,0.146094,0.276790,0.684783,0.739130,0.485577,0.40000
852,0.890514,0.901151,0.902244,0.894524,0.865666,0.805657,0.741799,0.916422,0.977738,0.785073,...,1.000000,1.000000,0.985392,1.000000,0.188730,0.154355,0.836957,0.858696,0.447115,0.60000


Split into train set and test set
* TrainSet : All the columns except the Qair ones
* Only the Qair columns

In [4]:
X = df[[col for col in df.columns if 'Qair' not in col]]
y = df[[col for col in df.columns if 'Qair_mean' in col]]

# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
y_test.columns

Index(['Fall_Qair_mean', 'Spring_Qair_mean', 'Summer_Qair_mean',
       'Winter_Qair_mean'],
      dtype='object')

### DT From Scratch

In [7]:
class DecisionTree:
    # Initialization of the DecisionTree class
    def __init__(self, max_depth=None, min_samples_split=2):
        '''  
            Stopping criteria : (because having too deep trees can cause overfitting)
                max_depth: Maximum depth of the tree, None means no limit on depth
                min_samples_split: Minimum number of samples required to split a node
        '''
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        # Initialize an empty list to store trees for each target variable in multi-output regression
        self.trees = []

    # Fit method to train the decision tree model on the data
    def fit(self, X, Y):
        # Loop through each target column in the Y dataset (multi-output regression)
        for target in range(Y.shape[1]):  # Y.shape[1] gives the number of target variables
            # Combine X and the current target column from Y to create a new dataset
            # This allows us to build a tree for each target
            tree = self._build_tree(np.c_[X, Y[:, target]], depth=0)
            # Append the generated tree for this target to the trees list
            self.trees.append(tree)

    # Recursive method to build a decision tree
    def _build_tree(self, data, depth):
        # Base case: stop building the tree if there are not enough samples to split or if max depth is reached
        if len(data) < self.min_samples_split or (self.max_depth and depth >= self.max_depth):
            # Return the mean of the target variable as the prediction (leaf node)
            return np.mean(data[:, -1])
        
        # Find the best split based on the current dataset
        best_split = self._find_best_split(data)
        # If no valid split is found, return the mean of the target variable (leaf node)
        if not best_split:
            return np.mean(data[:, -1])
        
        # Recursively build the left and right subtrees
        left = self._build_tree(best_split['left'], depth + 1)
        right = self._build_tree(best_split['right'], depth + 1)
        # Return a dictionary representing the tree node (split criteria and child nodes)
        return {'feature': best_split['feature'], 'threshold': best_split['threshold'], 'left': left, 'right': right}

    # Method to find the best feature and threshold to split the data
    def _find_best_split(self, data):
        best_split = None  # Variable to store the best split found
        min_error = float('inf')  # Start with an infinitely large error value
        
        # Loop through each feature in the dataset (excluding the target column)
        for feature in range(data.shape[1] - 1):
            # Get the unique values of the feature to try as possible thresholds
            thresholds = np.unique(data[:, feature])
            # Try each threshold value for the feature
            for threshold in thresholds:
                # Split the data into left and right subsets based on the threshold
                left = data[data[:, feature] <= threshold]
                right = data[data[:, feature] > threshold]
                # Skip splits where either side is empty
                if len(left) > 0 and len(right) > 0:
                    # Calculate the error of the current split
                    error = self._split_error(left, right)
                    # If this split has less error than previous ones, update the best split
                    if error < min_error:
                        min_error = error
                        best_split = {'feature': feature, 'threshold': threshold, 'left': left, 'right': right}
        # Return the best split found
        return best_split

    # Method to calculate the error of a split (MSE)
    def _split_error(self, left, right):
        left_error = np.mean((left[:, -1] - np.mean(left[:, -1])) ** 2) * len(left)
        right_error = np.mean((right[:, -1] - np.mean(right[:, -1])) ** 2) * len(right)
        return left_error + right_error


    # Predict method to generate predictions for input data X
    def predict(self, X):
        predictions = []  # List to store predictions for each target variable
        # Loop through each tree (one for each target variable)
        for tree in self.trees:
            # For each tree, make predictions for each row in X
            predictions.append(np.array([self._predict_row(row, tree) for row in X]))
        # Return the predictions for all trees as a column stack (multi-output predictions)
        return np.column_stack(predictions)

    # Recursive method to make a prediction for a single row of data
    def _predict_row(self, row, tree):
        # If the tree is a leaf node (not a dictionary), return the predicted value (mean of target values)
        if not isinstance(tree, dict):
            return tree
        # Otherwise, traverse the tree based on the feature value and threshold
        if row[tree['feature']] <= tree['threshold']:
            return self._predict_row(row, tree['left'])  # Go left
        else:
            return self._predict_row(row, tree['right'])  # Go right

    # To visualize the tree
    def visualize_tree(self, tree, feature_names):
        dot = graphviz.Digraph()
        self._add_node(dot, tree, feature_names, 0)
        return dot
    
    def _add_node(self, dot, tree, feature_names, node_id):
        if isinstance(tree, dict):
            feature_name = feature_names[tree['feature']]
            threshold = tree['threshold']
            dot.node(str(node_id), f'{feature_name} <= {threshold}')
            left_id = node_id * 2 + 1
            right_id = node_id * 2 + 2
            dot.edge(str(node_id), str(left_id), label="True")
            dot.edge(str(node_id), str(right_id), label="False")
            self._add_node(dot, tree['left'], feature_names, left_id)
            self._add_node(dot, tree['right'], feature_names, right_id)
        else:
            dot.node(str(node_id), f'Value: {tree:.2f}')
        return dot

* Evaluation

In [8]:
# Train and evaluate custom Decision Tree
start_time = time.time()
custom_tree = DecisionTree(max_depth=10, min_samples_split=10)
custom_tree.fit(X_train.to_numpy(), y_train.to_numpy())
y_pred_custom = custom_tree.predict(X_test.to_numpy())
custom_exec_time = time.time() - start_time

In [9]:
test_results = y_test.copy()
test_results['Fall_Qair_pred'] = y_pred_custom[:, 0]
test_results['Spring_Qair_pred'] = y_pred_custom[:, 1]
test_results['Summer_Qair_pred'] = y_pred_custom[:, 2]
test_results['Winter_Qair_pred'] = y_pred_custom[:, 3]
test_results

,Fall_Qair_mean,Spring_Qair_mean,Summer_Qair_mean,Winter_Qair_mean,Fall_Qair_pred,Spring_Qair_pred,Summer_Qair_pred,Winter_Qair_pred
66,0.145827,0.118276,0.228065,0.214708,0.136431,0.137320,0.289650,0.172665
434,0.241889,0.114588,0.090221,0.330721,0.232326,0.119180,0.113260,0.326452
198,0.329687,0.207926,0.137753,0.404552,0.259588,0.203869,0.149960,0.369193
212,0.112859,0.075347,0.063794,0.151454,0.225693,0.016553,0.113260,0.159675
792,0.663327,0.621103,0.671612,0.741401,0.643831,0.714755,0.467758,0.815022
...,...,...,...,...,...,...,...,...
211,0.068733,0.028131,0.037373,0.113032,0.181483,0.106427,0.113260,0.268338
259,0.297460,0.206505,0.097476,0.386294,0.259588,0.143831,0.086249,0.369193
7,0.151689,0.193434,0.361283,0.200099,0.149191,0.206170,0.437971,0.188288
281,0.181742,0.199617,0.089457,0.192295,0.152494,0.164292,0.121447,0.190706


In [19]:
# multioutput='uniform_average' ==> mse = (mse_target1 + mse_target2 + mse_target3 + mse_target4) / Number_of_targets 
# here Number_of_targets = 4 : Qair_fall, Qair_winter, Qair_spring, Qair_summer
custom_mse = mean_squared_error(y_test, y_pred_custom, multioutput='uniform_average')
custom_r2 = r2_score(y_test, y_pred_custom, multioutput='uniform_average')

# Print Results
print("\nCustom Decision Tree:")
print(f"MSE: {custom_mse:.4f}, R2: {custom_r2:.4f}, Time: {custom_exec_time:.4f} seconds")


Custom Decision Tree:
MSE: 0.0023, R2: 0.9232, Time: 19.8364 seconds


In [20]:
# Use multioutput= 'raw_values' if you want the individual values 
custom_mse = mean_squared_error(y_test, y_pred_custom, multioutput='raw_values')
custom_mse

array([0.00202611, 0.00347444, 0.0014067 , 0.002437  ])

In [ ]:
#  Visualize the first tree in the list
dot = custom_tree.visualize_tree(custom_tree.trees[0], X_train.columns)
dot.render("decision_tree", format="png", cleanup=True)
dot.view()  # Opens the pdf

'decision_tree.pdf'

### Sklearn DT

In [12]:
start_time = time.time()
sklearn_tree = DecisionTreeRegressor(max_depth=10, min_samples_split=10, random_state=42)
sklearn_tree.fit(X_train, y_train)
y_pred_sklearn = sklearn_tree.predict(X_test)
sklearn_exec_time = time.time() - start_time

In [13]:
sklearn_results = y_test.copy()
sklearn_results['Fall_Qair_pred'] = y_pred_sklearn[:, 0]
sklearn_results['Spring_Qair_pred'] = y_pred_sklearn[:, 1]
sklearn_results['Summer_Qair_pred'] = y_pred_sklearn[:, 2]
sklearn_results['Winter_Qair_pred'] = y_pred_sklearn[:, 3]
sklearn_results

,Fall_Qair_mean,Spring_Qair_mean,Summer_Qair_mean,Winter_Qair_mean,Fall_Qair_pred,Spring_Qair_pred,Summer_Qair_pred,Winter_Qair_pred
66,0.145827,0.118276,0.228065,0.214708,0.114479,0.100397,0.257045,0.182068
434,0.241889,0.114588,0.090221,0.330721,0.221688,0.109490,0.078630,0.324143
198,0.329687,0.207926,0.137753,0.404552,0.293366,0.202524,0.118764,0.376976
212,0.112859,0.075347,0.063794,0.151454,0.120843,0.047571,0.022094,0.169160
792,0.663327,0.621103,0.671612,0.741401,0.714732,0.679431,0.616566,0.747332
...,...,...,...,...,...,...,...,...
211,0.068733,0.028131,0.037373,0.113032,0.190708,0.100765,0.097634,0.268850
259,0.297460,0.206505,0.097476,0.386294,0.293366,0.202524,0.118764,0.376976
7,0.151689,0.193434,0.361283,0.200099,0.158349,0.210015,0.437971,0.197941
281,0.181742,0.199617,0.089457,0.192295,0.170335,0.142998,0.103240,0.204787


In [14]:
# Metrics for sklearn Decision Tree
sklearn_mse = mean_squared_error(y_test, y_pred_sklearn, multioutput='uniform_average')
sklearn_r2 = r2_score(y_test, y_pred_sklearn, multioutput='uniform_average')

print("\nSklearn Decision Tree:")
print(f"MSE: {sklearn_mse:.4f}, R2: {sklearn_r2:.4f}, Time: {sklearn_exec_time:.4f} seconds")


Sklearn Decision Tree:
MSE: 0.0020, R2: 0.9363, Time: 0.0187 seconds


In [23]:
sklearn_mse = mean_squared_error(y_test, y_pred_sklearn, multioutput='raw_values')
sklearn_mse

array([0.00196837, 0.00184458, 0.00158074, 0.00243126])